In [231]:
import numpy as np
import pandas as pd
import math
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD 
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.feature_extraction import DictVectorizer
from xgboost.sklearn import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn.ensemble import RandomForestRegressor

In [232]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import time
start = time.time()

meta_file = pd.read_csv("./AutoKaggle - Metadata.csv", encoding='cp1252',error_bad_lines=False)
arrOfRows = [64,360,239,316,515,518, 523]
nlp_rows = [239]
tabular_rows = [64,360,316,515,518]
row = 523
competition_name = meta_file['name'].loc[row]
print(meta_file['name'].loc[row])
dir_name = "./" + competition_name 
train_file = dir_name + "/data/train.csv"
row_file = dir_name + "/submission/row.csv"

meta = pd.read_csv(row_file)
print(meta['name'].loc[0])
def preprocessing(row):
   
    train = pd.read_csv(train_file)
#     find_row = meta.loc[row]
#     train = ''
#     test = None
    check_pred = False
    train_X = ''
    train_Y = ''
    test = None
    if meta['name'].loc[0] == 'kobe-bryant-shot-selection':
        train = pd.read_csv("./kobe-bryant-shot-selection/data/data.csv")
        check_pred = False
    elif meta['name'].loc[0] == 'mercedes-benz-greener-manufacturing':
        train = pd.read_csv("./mercedes-benz-greener-manufacturing/data/train.csv")
        test = pd.read_csv("./mercedes-benz-greener-manufacturing/data/test.csv")
      
    
#     if check_pred:
#         test = test.dropna()
    if meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        row = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv", encoding='cp1252')
        sms = train
        row_prepro = row['preprocessing function call'][0]
        prepro_ls = eval(row_prepro)
        sms = eval(prepro_ls[0])
        train = eval(prepro_ls[1])
        return train
    else:
        if type(meta['test set'].loc[0]) is str:
            row_file = dir_name + "/data/test.csv"
            test = pd.read_csv(row_file, encoding='cp1252', error_bad_lines=False)
        if type(meta['preprocessing function call'].loc[0]) is str:
            preprocessing_calls = eval(meta['preprocessing function call'].loc[0])
            for call in preprocessing_calls:
                exec(call)
                print(train.head())
        train = train.dropna()
        if type(meta["unwanted column"].loc[0]) is str:  # check if there's unwanted column
            column_list = eval(meta["unwanted column"].loc[0])
            train = train.drop(column_list,axis=1)
            print(train)
        if type(meta["numeric column"].loc[0]) is str:
            numeric=eval(meta["numeric column"].loc[0])
        for c in train.columns:
            if train[c].dtype == 'object':    # deal with text
                lbl = LabelEncoder() 
                if check_pred:
                    lbl.fit(list(train[c].values) + list(test[c].values)) 
                    train[c] = lbl.transform(list(train[c].values))
                    test[c] = lbl.transform(list(test[c].values))
                else:
                    lbl.fit(list(train[c].values))
                    train[c] = lbl.transform(list(train[c].values))
        target_name = str(meta['targetName'].loc[0])
        train_Y = train[[target_name]]
        train_X = train.drop(columns=target_name)
        train_X,test_X,train_Y,test_Y = train_test_split(train_X, train_Y, test_size=0.2)
        if check_pred:
            return train_X, train_Y, test_X, test_Y, pred_X
        else:
            return train_X, train_Y, test_X, test_Y,None#         if check_pred:


preprocessing(row)

rossmann-store-sales
rossmann-store-sales


/Users/Osborn/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3220: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  
   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2

[1017209 rows x 10 columns]


(         Store  DayOfWeek  Customers  Open  Promo  StateHoliday  \
 535320     906          7          0     0      0             0   
 912944     545          4        998     1      0             0   
 556266     667          2        871     1      1             0   
 147721     542          6        466     1      0             0   
 968820     671          3        512     1      0             0   
 738617     158          6        580     1      0             0   
 214573     494          2        995     1      0             0   
 622895     396          5       1664     1      1             0   
 224178      64          7          0     0      0             0   
 548591     797          2        319     1      0             0   
 251023     746          2        748     1      1             0   
 659003     824          1       1114     1      1             0   
 493074    1030          3        453     1      1             0   
 829722     948          2       2085     1     

In [233]:
def text_process(text):
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

def feature_extraction(row, X_train, X_test,X_pred):
    if meta['name'].loc[0] == 'uciml_sms-spam-collection-dataset':
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = rowcsv['featureExtractor function call'].loc[0]
        sms = X_train
        extract = eval(row_extract)
        sms['message'] = eval(extract[0])
        sms['message'] = eval(extract[1])
        text_feat = sms['message'].apply(str).copy()
        text_feat = eval(extract[2])
        vectorizer = eval(extract[3])
        features = eval(extract[4])
        features_train, features_test, labels_train, labels_test = train_test_split(features, sms['label'], test_size=0.3)
        return features_train, features_test, labels_train, labels_test
    else:
        if type(meta["featureExtractor function call"].loc[0]) is not str:
            print('not func')
            return X_train,X_test,X_pred
        extraction_function_calls = str(row_data["function call feature extraction"].loc[0])
        extraction_function_calls = extraction_function_calls.split(",")
        extraction_funtion_param = eval(row_data["function parameters feature extraction"].loc[0])
        function_nums = len(extraction_function_calls)
        for i in range(function_nums):
            str1 = extraction_function_calls[i]
            str2 = extraction_funtion_param[i]
            l_str = str1.split("(")
            l_str.insert(1,"("+str2)
            str_call = ''
            str_call = str_call.join(l_str)
            str_call = 'extractor' + '=' + str_call
            exec(str_call, globals(), globals())
            extracted_train = extractor.fit_transform(X_train)
            n_comp = extracted_train.shape[1]
            for j in range(0, n_comp):
                X_train['extractor'+ str(i)+"_"+str(j)] = extracted_train[:, j]
            if X_test is not None:
                extracted_test = extractor.fit_transform(X_test)
                for j in range(0, n_comp):
                    X_test['extractor'+ str(i)+"_"+str(j)] = extracted_test[:, j]
                return X_train,X_test
            else:
                return X_train,None

In [234]:
def feature_selection():
    pass

In [246]:
def estimation(row,X_train,X_test,Y_train, Y_test):
    if row in nlp_rows:
        rowcsv = pd.read_csv("./uciml_sms-spam-collection-dataset/submission/row.csv")
        row_extract = eval(rowcsv['estimator1 function call'].loc[0])
        mnb = eval(row_extract[0])
        eval(row_extract[1])
        pred = eval(row_extract[2])
        if rowcsv['performanceMetric'].loc[0] == 'accuracy':
            return accuracy_score(Y_test, pred)
    else:
        estimation_function_calls = eval(meta["estimator1 function call"].loc[0])
        print(estimation_function_calls)
        if len(estimation_function_calls) == 1:
            str_call = estimation_function_calls[0]
            str_call = 'estimator' + '=' + str_call
            exec(str_call,globals(),globals())
            
            if meta["taskType"].loc[0] == 'classification':
                estimator.fit(X_train,Y_train)
                Y_pred = estimator.predict(X_test)
                print('here')
                print(accuracy_score(Y_test,Y_pred))
            elif meta["taskType"].loc[0] == 'regression':
                estimator.fit(X_train, Y_train)
                Y_pred = estimator.predict(X_test)
                print("Mean Squared Error is: ", mean_squared_error(Y_test,Y_pred))
        else:
            estimators = []
            n_estimators = len(estimation_function_calls)
            for i in range(n_estimators):
                str1 = extraction_function_calls
                l_str = str1.split("(")
                l_str.insert(1,"("+str2)
                str_call = ''
                str_call = str_call.join(l_str)
                str_call = 'estimator' + '=' + str_call
                print(l_str)
                print(str_call)
                exec(str_call)
                estimators.append(estimator)
                postprocessing(estimators,stack = True)



In [247]:
if row in nlp_rows:
    train_set = preprocessing(row)
    X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
    Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
    print(Y_pred)
if row in tabular_rows:
    X_train,Y_train,X_test,Y_test,X_pred = preprocessing(row)
    X_train,X_test,X_pred = feature_extraction(row,X_train,X_test,X_pred)
    estimation(row,X_train,X_test,Y_train, Y_test)

In [248]:
# if meta['name'].loc[row] == 'uciml_sms-spam-collection-dataset':
#     train_set = preprocessing(row)
#     X_train, X_test, Y_train, Y_test = feature_extraction(row, train_set, None)
#     Y_pred = estimation(row, X_train, X_test, Y_train, Y_test)
#     print(Y_pred)

In [249]:
def postprocessing(estimators,stack):
    pass

In [250]:
end = time.time()
print("Use", end - start, "seconds to run this.")

Use 142.54396104812622 seconds to run this.


In [251]:
meta

,Unnamed: 0,NetId,name,URL,endDate,dataSize,table,image,audio,video,...,Unnamed: 212,Unnamed: 213,Unnamed: 214,Unnamed: 215,Unnamed: 216,Unnamed: 217,Unnamed: 218,Unnamed: 219,Unnamed: 220,Unnamed: 221
0,524,"jc7483, zl1596",rossmann-store-sales,https://www.kaggle.com/c/rossmann-store-sales,12/14/15 23:59,39.55MB,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [252]:
meta['test set'].loc[0]

'rossmann-store-sales'

In [253]:
A = preprocessing(row)

/Users/Osborn/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  
   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2

[1017209 rows x 10 columns]


In [254]:
train_X, train_Y, test_X, test_Y, pred = preprocessing(row)

   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  
   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
3      4          5 2015-07-31  13995       1498     1      1            0   
4      5          5 2

[1017209 rows x 10 columns]


In [255]:
train_Y

,Sales
891841,5343
304053,9125
336962,12264
134058,17638
52274,11608
588056,6922
626627,10998
50220,6652
284985,11155
132666,8701


In [256]:
estimation(row,train_X, test_X, train_Y, test_Y)

['RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)']
RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
estimator=RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)


/Users/Osborn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Mean Squared Error is:  3979675.8382824594
